In [ ]:
!pip install pyshp

In [ ]:
import ee
import geemap


In [ ]:
ee.Authenticate(force=False)
ee.Initialize(project='modern-rex-429509-a6')


In [ ]:
# Create a new map
Map = geemap.Map()



In [ ]:
# Set the center of the map
Map.setCenter(6.5, 6.5, 7)


In [ ]:
# Define the geographic coordinates of Edo state
edo_bounds = ee.Geometry.Rectangle([5.5, 6.2, 6.8, 7.2])


In [ ]:
# Filter the COPERNICUS/S2 image collection by the Edo state bounds
data = ee.ImageCollection("COPERNICUS/S2").filterBounds(edo_bounds)


In [ ]:
# Filter the image collection by date
data = data.filterDate("2019-08-01", "2019-12-31")


In [ ]:
# Filter the image collection by cloud coverage
data = data.filter(ee.Filter.lte('CLOUD_COVERAGE_ASSESSMENT',5))



In [ ]:
# Mosaic multiple images
image = ee.Image(data.mosaic())


In [ ]:
# Calculate NDVI
ndvi = image.expression("(NIR - RED) / (NIR + RED)", {
  "NIR": image.select("B8"),
  "RED": image.select("B4")
})



In [ ]:
# Load the Edo shapefile
edo_shapefile = ee.FeatureCollection('projects/modern-rex-429509-a6/assets/Edo_LGA')

In [ ]:
# Clip the NDVI image to Edo state
ndvi_edo = ndvi.clip(edo_shapefile)

In [ ]:
display={
    "min":0,
    "max":1,
    "palette":[ 'red','orange', 'yellowgreen','green', 'green','green']
}

In [ ]:
# Load the water bodies layer
water_bodies = ee.FeatureCollection('projects/modern-rex-429509-a6/assets/Water_body')

# Display settings for water bodies
display_water_bodies = {
  "color": "blue",
  "pointSize": 1
}


In [ ]:

# Add the clipped NDVI layer to the map
Map.addLayer(ndvi_edo, display)

In [ ]:
# Add the water bodies layer to the map
Map.addLayer(water_bodies, display_water_bodies)


In [ ]:
Map

Map(center=[6.5, 6.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…